#### Imports

In [1]:
# Telegram Bot
import telebot
import difflib
from telebot.types import InlineKeyboardMarkup, InlineKeyboardButton
from datetime import datetime, timedelta

import re
import spacy

from ingressos.ibge import get_estados_cidades
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Config Driver
from config.config_driver import ConfigDriver


#### Variaveis

In [2]:
API_TOKEN = '7135171388:AAExkHE82hR68W21q-KJCsxpJmVU9gtLNPA'
bot = telebot.TeleBot(API_TOKEN)

config_driver = ConfigDriver()
driver = config_driver.get_driver()

saudacoes = ['oi', 'olá', 'ola', 'eae', 'e aí', 'oiie', 'oie, tudo bem?', 'oi, tudo bem?', 'oi, como vai?', 'oi, como você está?']


#### Main

In [3]:
# Variavél global de cinemas
cinemas = None

# Carregar o modelo de linguagem português do spaCy
nlp = spacy.load("pt_core_news_sm")

def process_natural_language(message):
    user_text = message.text.lower().strip()
    doc = nlp(user_text)
    
    intent = None
    cidade = None
    date = None

    if any(token.lemma_ in ["cinema", "procurar", "buscar", "filme", "assistir"] for token in doc):
        intent = "cinema"
    
    city_match = re.search(r"em (.+)", user_text)
    if city_match:
        cidade = city_match.group(1).strip()
    
    if any(token.lemma_ in ["hoje"] for token in doc):
        date = datetime.today().strftime("%d/%m")
        
    if any(token.lemma_ in ["amanhã"] for token in doc):
        date = (datetime.today() + timedelta(days=1)).strftime("%d/%m")
    
    
    # Executando a ação com base na intenção e na cidade
    if intent == "cinema" and cidade:
        process_city_name(message, filmes=False, city=cidade, date=date)
    elif intent == "cinema":
        handle_cinema_markup(message)
    elif any(token.lemma_ in ["olá", "oi", "ajuda", "bom"] for token in doc):
        handle_first_contact(message)
    else:
        bot.send_message(message.chat.id, "Desculpe, não entendi. Tente frases como 'quero ver cinemas em [cidade]' ou 'buscar filmes em [cidade]'.")

# Inicialização do bot
@bot.message_handler(func=lambda message: True)
def reply_message(message):
    # Processa o texto para identificar a intenção
    process_natural_language(message)

# Menu inicial do bot
def handle_first_contact(message):
    bot.send_message(message.chat.id, "Olá, eu sou o bot do Ingresso.com, como posso te ajudar")

# Caminho de cinema selecionado
def handle_cinema_markup(message):
    # Recebe o estado enviado
    state_msg = bot.send_message(message.chat.id, "Digite a cidade do cinema que deseja buscar")
    bot.register_next_step_handler(state_msg, process_city_name)


def process_city_name(message, filmes=False, city=None, date=None):
    if city is None:
        city = message.text.strip()
        
    print(date)
        

    estados_cidades = get_estados_cidades()
    todas_cidades = [cidade for cidades in estados_cidades.values() for cidade in cidades]
    closest_matches = difflib.get_close_matches(city, todas_cidades, n=1, cutoff=0.6)

    if closest_matches:
        city = closest_matches[0]
        bot.send_message(message.chat.id, f"Buscando cinemas em {city}...")

        formatted_city = city.replace(" ", "-").lower()
        linkWebDriver = f"https://www.ingresso.com/cinemas?city={formatted_city}"
        
        if filmes:
            linkWebDriver = f"https://www.ingresso.com/filmes?city={formatted_city}"

        driver.get(linkWebDriver)

        try:
            fecharPopup = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
            )
            if fecharPopup.is_displayed():
                fecharPopup.click()
        except Exception:
            print("Popup button not found, continuing without clicking.")

        if not filmes:
            getCinemasFromCity(driver, message, date)
    else:
        bot.send_message(message.chat.id, "Nenhuma cidade correspondente encontrada. Tente novamente.")
        handle_cinema_markup(message)
    
# A partir de uma cidade encontrada, busca os cinemas e envia para o usuário escolher
def getCinemasFromCity(driver , message, date):
    try:
        fecharPopup = WebDriverWait(driver, 99).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if(fecharPopup.is_displayed()):
            fecharPopup.click()
    except Exception:
        print("Popup button not found, continuing without clicking.")
    
    global cinemas 
    cinemas = driver.find_elements(By.CSS_SELECTOR, "h3.text-base.leading-4")  

    # Print the names of the cinemas for debugging purposes
    for cinema in cinemas:
        print(cinema.text)

    # Criar um teclado inline
    markup = InlineKeyboardMarkup()

    # Adicionar botões ao teclado com o texto do cinema e dados de callback
    for index, cinema in enumerate(cinemas):
        markup.add(InlineKeyboardButton(cinema.text, callback_data="ERIKAOXD_" + str(index) + "_" + date))
    # Enviar o teclado inline para o usuário
    bot.send_message(message.chat.id, "Cinemas:", reply_markup=markup)

# Com o cinema selecionado busca os filmes disponíveis
def getFilmesByCinema(driver, cinemaSelecionado, message, date=None):    
    global cinemas
    
    print(date)
    
    try:
        fecharPopup = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "accept-btn"))
        )
        if(fecharPopup.is_displayed()):
            fecharPopup.click()
    except Exception:
        print("Popup button not found, continuing without clicking.")
    
    # Seleciona o primeiro cinema (pode ser ajustado)
    cinema_element = cinemas[cinemaSelecionado].find_element(By.XPATH, '..')

    # Rolagem para o cinema selecionado
    driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", cinema_element)
    
    time.sleep(2)
    # Espera até que o elemento esteja visível e clicável
    WebDriverWait(driver, 10).until(EC.visibility_of(cinema_element))
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(cinema_element))

    cinema_element.click()

    if date == None:
        date = datetime.today().strftime("%d/%m")
    
    try:
        diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{date}')]")
    except Exception:
        time.sleep(3)
        try:
            diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{date}')]")
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", diaEscolhidoElemento)
        except Exception:
            print(f"Element still not found for date: {date} after retrying")
    
    
    try:
        cookie_consent = driver.find_element(By.CLASS_NAME, "CookieConsent")
        if cookie_consent.is_displayed():
            close_button = cookie_consent.find_element(By.TAG_NAME, "button")
            close_button.click()
    except Exception:
        print("Cookie consent not found, continuing without clicking.")

    # Now attempt to click the desired element
    try:
        diaEscolhidoElemento = driver.find_element(By.XPATH, f"//span[contains(text(), '{date}')]")
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", diaEscolhidoElemento)
        diaEscolhidoElemento.click()
    except Exception:
        print("Element click intercepted, please check the page layout.")


    # Captura os blocos de filmes
    blocos_filmes = driver.find_elements(By.CSS_SELECTOR, '.relative.my-5')
    
    
    # Itera sobre cada bloco para extrair o nome do filme, horários e links
    for bloco in blocos_filmes:
        horarios_filmes = ""
        # Extrai o nome do filme
        nome_filme = bloco.find_element(By.CSS_SELECTOR, "h3 a").text
        
        # Extrai os elementos <a> que contêm os horários
        horarios_elements = bloco.find_elements(By.CSS_SELECTOR, "a[href*='sessionId']")
        
        # Extrai o horário e o link
        horarios = []
        for horario_element in horarios_elements:
            horario = horario_element.find_element(By.CSS_SELECTOR, "span").text  # Extrai o texto do horário
            link = horario_element.get_attribute("href")  # Extrai o link
            horarios.append((horario, link))  # Adiciona à lista de horários

        imgFilme = bloco.find_element(By.CSS_SELECTOR, "img").get_attribute("src")

        # Append the movie name to the message
        horarios_filmes += f"*Nome do Filme:* {nome_filme}\n"
        horarios_filmes += f"*Horários:*\n"
        
        # Append each horario and link to the horarios_filmes
        for horario, link in horarios:
            horarios_filmes += f"  - [{horario}]({link})\n"
        
        horarios_filmes += f"---\n"
        
        bot.send_photo(message.chat.id, imgFilme, parse_mode='MarkdownV2')
        bot.send_message(message.chat.id, horarios_filmes)



        
@bot.callback_query_handler(func=lambda call: True)
def callback_query(call):
    if call.data.startswith("ERIKAOXD"):
        index = int(call.data.split("_")[1])
        date = call.data.split("_")[2] if len(call.data.split("_")) > 2 else None
        getFilmesByCinema(driver, index, message=call.message, date=date)
    else:
        # Outros callbacks para "cinema" ou "filme"
        if call.data == "cinema":
            handle_cinema_markup(call.message)


#### Run Bot

In [ ]:
bot.polling()

09/11
Popup button not found, continuing without clicking.
Cinemark Center Vale
Cinemark Colinas
Cinépolis Jardim Oriente
Kinoplex Diamante
Kinoplex Vale Sul
09/11
Popup button not found, continuing without clicking.
